- Pytorch 기반 
- MLP 학습 및 평가. SGD(mini batch) 기반 학습
- S-SAE 학습 및 평가. optimizer 와 scheduler 까지 활용. 
    - SAE 먼저 pretrain 하고, encoder 만 떼어냄
    - pretrain된 encoder 에 regressor 붙여서 재학습(fine-tuning) 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import skew
from catboost import CatBoostRegressor
import gc
from sklearn.linear_model import Ridge , LogisticRegression
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
import xgboost as xgb
import time
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [41]:
# 데이터 로딩,전처리

# 데이터 불러오기 
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)
file2 = r"cycletime_minmax.csv"
ct_df = pd.read_csv(file2)
ct_df = ct_df.sample(frac=0.01, random_state=42)

# x-y 분리, y(타겟) 정규화
y_ct = ct_df.iloc[:,-1]
x_ct = ct_df.iloc[:,:-1]
y_ct = np.log1p(y_ct)

# 숫자형/범주형 변수 분리 
numerical_list=[]
categorical_list=[]

for i in x_ct.columns :
  if x_ct[i].dtypes == 'O' :
    categorical_list.append(i)
  else :
    numerical_list.append(i)

# 범주형 변수 원핫인코딩 
x_ct = pd.get_dummies(x_ct, columns=categorical_list, drop_first=True, dtype=int)

# 숫자형변수 float32로 통일
for col in x_ct.select_dtypes(include=['float64']).columns:
    x_ct[col] = x_ct[col].astype('float32')

딥러닝 모델
1. MLP
2. S-SAE
3. (AE) + FCM + BPN 
4. PCA + K-means + BPN 

1. MLP

In [42]:
# MLP 

class MLP(torch.nn.Module):
    def __init__(self, input_size, dropout_prob=0.2):
        super(MLP, self).__init__()
        
        # Initialize layers
        self.linear_1 = torch.nn.Linear(input_size, 600)
        self.batch_norm1 = torch.nn.BatchNorm1d(600)

        self.linear_2 = torch.nn.Linear(600, 150)
        self.batch_norm2 = torch.nn.BatchNorm1d(150)
        
        self.linear_out = torch.nn.Linear(150, 1)
        
        # Activation functions
        self.leaky_relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(dropout_prob)

        # Initialize weights using Xavier (Glorot) initialization
        torch.nn.init.xavier_uniform_(self.linear_1.weight)
        torch.nn.init.xavier_uniform_(self.linear_2.weight)
        torch.nn.init.xavier_uniform_(self.linear_out.weight)

    def forward(self, input_tensor):
        x = self.dropout(self.leaky_relu(self.batch_norm1(self.linear_1(input_tensor))))
        x = self.dropout(self.leaky_relu(self.batch_norm2(self.linear_2(x))))
        output = self.linear_out(x)
        return output

In [43]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE,USE_CUDA)

cpu False


In [44]:
# 모델 생성
input_dim = x_ct.shape[1]
input_dim

644

In [45]:
# 하이퍼 파라미터 세팅 
epochs = 20
learning_rate = 0.1
BATCH_SIZE = 256


In [46]:
import torch
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from torch.utils.data import DataLoader, TensorDataset

In [47]:
# 데이터를 PyTorch 텐서로 변환
X_tensor = torch.tensor(x_ct.values, dtype=torch.float32).to(DEVICE)
Y_tensor = torch.tensor(y_ct.values, dtype=torch.float32).to(DEVICE).view(-1, 1)

In [48]:
X_tensor

tensor([[0.2917, 1.0000, 0.9300,  ..., 0.0000, 1.0000, 0.0000],
        [0.8750, 0.2500, 0.5200,  ..., 1.0000, 0.0000, 0.0000],
        [0.6667, 1.0000, 0.8400,  ..., 0.0000, 1.0000, 0.0000],
        ...,
        [0.5833, 0.0000, 0.6800,  ..., 0.0000, 0.0000, 1.0000],
        [0.5833, 0.0000, 0.4300,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 0.2500, 0.0900,  ..., 1.0000, 0.0000, 0.0000]])

In [49]:
# 데이터를 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X_tensor, Y_tensor, test_size=0.2, random_state=42)

# 데이터 로더 생성
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [31]:
print(type(X_tensor), type(train_dataset), type(train_loader))

<class 'torch.Tensor'> <class 'torch.utils.data.dataset.TensorDataset'> <class 'torch.utils.data.dataloader.DataLoader'>


In [35]:
print(X_train.shape, len(train_loader.dataset))

torch.Size([417423, 694]) 417423


In [54]:
# 모델 초기화 및 설정
mlp = MLP(input_size=X_train.shape[1]).to(DEVICE)
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)

# 훈련 루프
mlp.train()
for epoch in range(5):  # or your choice of number of epochs
    running_loss = 0.0
    for inputs, targets in train_loader:  # train dataset 6400개 이고, batch size가 64라면 -> 100번의 iteration
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        optimizer.zero_grad()
        predictions = mlp(inputs)
        loss = F.mse_loss(predictions, targets)  # 64개 데이터에 대한 loss값은 1개 ! 
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        # 아래 len(train_loader.dataset) 으로 나누기위해 inputs.size(0) 즉, batch size를 곱해주는 것. 
        # inputs.size(0)을 안곱하면 아래 나눌떄는 len(train_loader.dataset) / inputs.size(0) 만큼을 곱해줄 것. 


    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")


Epoch 1, Loss: 3.2395
Epoch 2, Loss: 0.7638
Epoch 3, Loss: 0.6492
Epoch 4, Loss: 0.5780
Epoch 5, Loss: 0.5245


In [55]:
y_test.cpu().device

device(type='cpu')

In [18]:
import csv as csv

# 평가 루프
mlp.eval()
with torch.no_grad():
    X_test = X_test.to(DEVICE)
    predictions = mlp(X_test).cpu()
    y_test = y_test.cpu()
    individual_mses = (predictions - y_test) ** 2
    individual_mapes = torch.abs((predictions - y_test) / y_test) * 100

# CSV 파일로 저장
def save_to_csv(mses, mapes, filename="evaluation_results_MLP.csv"):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["MSE", "MAPE"])
        for mse, mape in zip(mses.numpy().flatten(), mapes.numpy().flatten()):
            writer.writerow([mse, mape])

save_to_csv(individual_mses, individual_mapes)
print("MSE and MAPE values have been saved to test_results.csv.")

MSE and MAPE values have been saved to test_results.csv.


2. S-SAE

In [19]:
X_tensor.shape[1]

694

In [58]:
# S-SAE 사전학습 

class SparseAutoencoder(nn.Module):
    def __init__(self, input_size):
        super(SparseAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 600),
            nn.ReLU(),
            nn.Linear(600, 300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, 40)  # Bottleneck
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(40, 100),
            nn.ReLU(),
            nn.Linear(100, 300),
            nn.ReLU(),
            nn.Linear(300, 600),
            nn.ReLU(),
            nn.Linear(600, input_size)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def l1_penalty(self, beta):
        l1_norm = sum(p.abs().sum() for p in self.encoder.parameters())
        return beta * l1_norm

# Model instantiation
input_size = X_tensor.shape[1] 
SAE = SparseAutoencoder(input_size).to(DEVICE)

In [59]:
class SAEReg(nn.Module):
    def __init__(self, encoder, output_size=1):
        super(SAEReg, self).__init__()
        self.encoder = encoder
        # Assuming the encoder's output size (bottleneck) is 40
        self.regressor = nn.Sequential(
            nn.Linear(40, output_size)  # Output layer for regression
        )

    def forward(self, x):
        encoded = self.encoder(x)
        regression_output = self.regressor(encoded)
        return regression_output


In [63]:
SAE.load_state_dict(torch.load('ssae.pth'))
# Extract the encoder part
encoder_part = SAE.encoder
# Create the SAE with regression model
SAEREG = SAEReg(encoder=encoder_part).to(DEVICE)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [30]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_tensor, Y_tensor, test_size=0.2, random_state=42)

# Data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

# Optimizer and learning schedule
optimizer = torch.optim.Adam(SAEREG.parameters(), lr=0.01, weight_decay=1e-7)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=74, gamma=0.5)

In [25]:
epochs = 25
learning_rate = 0.07
BATCH_SIZE = 256

In [31]:
for epoch in range(epochs):
    SAEREG.train()
    running_loss = 0.0 
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        optimizer.zero_grad()
        predictions = SAEREG(inputs)
        loss = nn.functional.mse_loss(predictions.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    scheduler.step()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")


C:\Users\sbs\AppData\Local\Temp\ipykernel_24804\3721360348.py:8: UserWarning: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.mse_loss(predictions.squeeze(), targets)
C:\Users\sbs\AppData\Local\Temp\ipykernel_24804\3721360348.py:8: UserWarning: Using a target size (torch.Size([143, 1])) that is different to the input size (torch.Size([143])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.mse_loss(predictions.squeeze(), targets)


Epoch 1, Loss: 0.7703
Epoch 2, Loss: 0.7703
Epoch 3, Loss: 0.7699
Epoch 4, Loss: 0.7701
Epoch 5, Loss: 0.7698
Epoch 6, Loss: 0.7696
Epoch 7, Loss: 0.7696
Epoch 8, Loss: 0.7693
Epoch 9, Loss: 0.7696
Epoch 10, Loss: 0.7693
Epoch 11, Loss: 0.7689
Epoch 12, Loss: 0.7688
Epoch 13, Loss: 0.7691
Epoch 14, Loss: 0.7689
Epoch 15, Loss: 0.7687
Epoch 16, Loss: 0.7688
Epoch 17, Loss: 0.7685
Epoch 18, Loss: 0.7686
Epoch 19, Loss: 0.7687
Epoch 20, Loss: 0.7684
Epoch 21, Loss: 0.7686
Epoch 22, Loss: 0.7685
Epoch 23, Loss: 0.7684
Epoch 24, Loss: 0.7682
Epoch 25, Loss: 0.7684


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [32]:
# Evaluation on test data
SAEREG.eval()
with torch.no_grad():
    X_test = X_test.to(DEVICE)
    predictions = SAEREG(X_test).cpu()
    y_test = y_test.cpu()
    mse = (predictions - y_test) ** 2
    mape = torch.abs((predictions - y_test) / y_test) * 100

# Save the results to CSV
def save_to_csv(mses, mapes, filename="evaluation_results_SSAE.csv"):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["MSE", "MAPE"])
        for mse, mape in zip(mses.numpy().flatten(), mapes.numpy().flatten()):
            writer.writerow([mse, mape])

save_to_csv(mse, mape)
print("MSE and MAPE values have been saved to test_results.csv.")

MSE and MAPE values have been saved to test_results.csv.
